<a href="https://colab.research.google.com/github/MOSSAWIII/Feature-Selection/blob/main/SimulatedAnnealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score, recall_score, precision_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def simulated_annealing_cv(X, y, model, iterations, restart_threshold, initial_size=0.5, n_splits=10):
  """
  Performs simulated annealing for feature selection with stratified cross-validation and returns best features, best AUC ROC score, sensitivity, specificity, and standard deviation of the best score.

  Args:
      X (np.array): Feature matrix.
      y (np.array): Target labels (CASE or CTRL).
      model (object): A scikit-learn compatible model object.
      iterations (int): Number of iterations for the simulated annealing process.
      restart_threshold (int): Threshold for restarting the search.
      initial_size (float): Proportion of features to include in the initial subset.
      n_splits (int): Number of splits for stratified cross-validation.

  Returns:
      tuple: (best_features, best_score, best_sensitivity, best_specificity, score_std)
  """

  skf = StratifiedKFold(n_splits=n_splits)
  best_features = None
  best_scores = []
  best_sensitivities = []
  best_specificities = []
  best_params = None

  for train_index, val_index in skf.split(X, y):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Initialize features
    num_features = X.shape[1]
    current_features = np.random.choice(num_features, int(num_features * initial_size), replace=False)
    best_features_fold = current_features
    best_score_fold = 0
    best_params_fold = None
    best_sensitivity_fold = 0
    best_specificity_fold = 0
    iteration = 0
    no_improvement_count = 0

    while iteration < iterations:
      # Perturb features
      perturbed_features = current_features.copy()
      to_change = np.random.randint(0, len(perturbed_features))
      if np.random.rand() < 0.5:
        # Add a feature
        candidate_feature = np.random.choice(list(set(range(num_features)) - set(perturbed_features)))
        perturbed_features = np.append(perturbed_features, candidate_feature)
      else:
        # Remove a feature
        perturbed_features = np.delete(perturbed_features, to_change)

      # Evaluate candidate model
      model.fit(X_train[:, perturbed_features], y_train)
      y_pred = model.predict(X_val[:, perturbed_features])
      score = roc_auc_score(y_val, y_pred)
      sensitivity = recall_score(y_val, y_pred)
      specificity = recall_score(y_val, y_pred, pos_label=0)

      # Print metrics for information (not used for decision making)
      print(f"Iteration {iteration}, Features: {len(perturbed_features)}, ROC_AUC: {score:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}")

      # Acceptance probability (based on ROC_AUC only)
      if score > best_score_fold:
        best_score_fold = score
        best_features_fold = perturbed_features
        best_params_fold = model.get_params()
        best_sensitivity_fold = sensitivity
        best_specificity_fold = specificity
        no_improvement_count = 0
      else:
        acceptance_probability = np.exp((score - best_score_fold) / (iteration + 1))
        if np.random.rand() < acceptance_probability:
          current_features = perturbed_features
        no_improvement_count += 1

      # Restart if necessary
      if no_improvement_count >= restart_threshold:
        current_features = best_features_fold.copy()
        no_improvement_count = 0

      iteration += 1

    # Store best scores, sensitivities, and specificities for each fold
    best_scores.append(best_score_fold)
    best_sensitivities.append(best_sensitivity_fold)
    best_specificities.append(best_specificity_fold)

    # Update best features and best params
    if best_features is None:
      best_features = best_features_fold
      best_params = best_params_fold  # Store best params from the first fold initially
    else:
      best_features = np.logical_or(best_features, best_features_fold)
      if best_score_fold > best_score:  # Update best_params if current fold has a better score
        best_params = best_params_fold

  best_score = np.mean(best_scores)
  best_sensitivity = np.mean(best_sensitivities)
  best_specificity = np.mean(best_specificities)
  score_std = np.std(best_scores)
  return best_features, best_score, best_sensitivity, best_specificity, score_std, best_params


In [ ]:
# Function for Simulated Annealing with Logistic Regression
def SA_LogisticRegression(X, y, iterations, restart_threshold, initial_size=0.5, n_splits=10):
  model = LogisticRegression()
  best_features, best_score, best_sensitivity, best_specificity, score_std, best_params = simulated_annealing_cv(X, y, model, iterations, restart_threshold, initial_size, n_splits)
  return best_features, best_score, best_sensitivity, best_specificity, score_std, best_params

# Function for Simulated Annealing with Random Forest
def SA_RandomForest(X, y, iterations, restart_threshold, initial_size=0.5, n_splits=10):
  model = RandomForestClassifier()
  best_features, best_score, best_sensitivity, best_specificity, score_std, best_params = simulated_annealing_cv(X, y, model, iterations, restart_threshold, initial_size, n_splits)
  return best_features, best_score, best_sensitivity, best_specificity, score_std, best_params